# Real Estate Profit Opportunity Identification For Improvement

## Step_1: Data Wrangling

Normally, at least at this level of training, data wrangling involves cleaning up text. But we are attempting something more challenging and sophisticated - wrangling *images*.

To do so, we start with this template with packages and tools such as TensorFlow (paired with keras), cv2 & glob.

## 1. Import Packages

In [1]:
# Import the necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:
import warnings

warnings.filterwarnings(action = 'ignore')

import os
import sys

import cv2
import glob
import skvideo.io


In [9]:
#have to divide up these imports because taking forever so need to see what the holdup is

import tensorflow as tf


from tensorflow import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image   # for preprocessing the images
from tensorflow.keras.utils import to_categorical #np_utils
from tensorflow.keras import layers
from tensorflow.keras.layers import (Flatten, Dense, Activation, MaxPooling2D, Conv2D, InputLayer)
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD

#oh nice!!! finally got all of it!

In [15]:
from numba import double, jit, njit, vectorize

import progressbar

import time

import PIL
from PIL import Image, ImageEnhance

import math

from sklearn.metrics import (confusion_matrix, classification_report, ConfusionMatrixDisplay)
from sklearn.utils import shuffle

import pickle

from skimage.transform import resize   # for resizing images

from moviepy.editor import VideoFileClip, concatenate_videoclips

## 2. Data Wrangling

In [ ]:
# Establish the path to read all of the images 

#so i have a WEBSITE/url (github) w/ the pics - so use scrapy?
#don't know if i can download images but would prob take up way too much space anyway unnecessarily

path_training_flip = glob.glob('/Users/arnaldofolder/Documents/Apziva/Fourth Project/images/training/flip/*.jpg')

path_training_notflip = glob.glob('/Users/arnaldofolder/Documents/Apziva/Fourth Project/images/training/notflip/*.jpg')

path_testing_flip = glob.glob('/Users/arnaldofolder/Documents/Apziva/Fourth Project/images/testing/flip/*.jpg')

path_testing_notflip = glob.glob('/Users/arnaldofolder/Documents/Apziva/Fourth Project/images/testing/notflip/*.jpg')

## 3. Data Preprocessing

In [ ]:
# Define a function that will do all the preprocessing for each image to be ready for modeling

def image_preprocessing(path):
    # Create an empty list to store all the preprocessed images
    images = []
    # Start by creating a for loop through all the path and make the preprocessing to each image
    for i in path:
        # Firstly read the image
        img = cv2.imread(i)
        # Adjust the size so all iamges will have the same size
        img = cv2.resize(img, dsize = (70,140), interpolation=cv2.INTER_CUBIC)
        # Crop to remove part of the images I don't need for the modeling part
        y,h,x,w = 0,100,0,70
        img = img[y:y+h, x:x+w]
        # Adjust brightness, contrast
        alpha=1.5
        beta=0.5
        img = cv2.addWeighted(img, alpha, np.zeros(img.shape, img.dtype), 0, beta)
        # Normalize the images to be black and white by reverting the images and then dividing by 255.0
        img = cv2.bitwise_not(img)
        img = img/255

        # Append the img to the list images
        images.append(img)
        # Create the video

    # Return the list with the preprocessed images
    return images

In [ ]:
# Define functions to create a video from images

def video_creator(path, pathIn, time, fps):
    frame_video = []
    for i in path:
        img = cv2.imread(i)
        height, width, layers = img.shape
        size = (width, height)
        frame_video.append(img)
    out = cv2.VideoWriter(pathIn, cv2.VideoWriter_fourcc(*'mp4v'), fps, size)
    for i in range(len(path)):
        out.write(frame_video[i])
    
    out.release()

In [ ]:
# Read and preprocess the training data

img_training_flip = image_preprocessing(path = path_training_flip)

# Read the training not flip

img_training_notflip = image_preprocessing(path = path_training_notflip)

# Read the test flip

img_testing_flip = image_preprocessing(path = path_testing_flip)

# Read the test not flip

img_testing_notflip = image_preprocessing(path = path_testing_notflip)

In [ ]:
# Define the labels for the problem

y_train_flip = [1 for i in range(0, len(img_training_flip))]

y_train_notflip = [0 for i in range(0, len(img_training_notflip))]

y_test_flip = [1 for i in range(0, len(img_testing_flip))]

y_test_notflip = [0 for i in range(0, len(img_testing_notflip))]

In [ ]:
# Create the video for the training flip

video_creator(path = path_training_flip, pathIn = 'training_flip.avi', time = len(path_training_flip), fps = 1)

# Create the video for the training not flip

video_creator(path = path_training_notflip, pathIn = 'training_notflip.avi', time = len(img_training_notflip), fps = 1)

# Create the video for the test flip

video_creator(path = path_testing_flip, pathIn = 'test_flip.avi', time = len(img_testing_flip), fps = 1)

# Create the video for the test not flip

video_creator(path = path_testing_notflip, pathIn = 'test_notflip.avi', time = len(img_testing_notflip), fps = 1)



### Modeling

In [ ]:
# Define the X_train, X_test, y_train and y_test for analysis

X_train = np.concatenate((img_training_flip, img_training_notflip), axis = 0)

X_test = np.concatenate((img_testing_flip, img_testing_notflip), axis = 0)

y_train = np.append(y_train_flip, y_train_notflip)

y_test = np.append(y_test_flip, y_test_notflip)

In [ ]:
# See if the shapes matches between the X_trian and y_train and the X_test and y_test

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# Create a new array that will have the original arrays (labels and values) but they will be shuffled. 

# Create the array for the train data set

X_train_shuffle = []

# It is necessary to create a for loop with enumeration as well
for i,j in enumerate(X_train):
    # The new array would be the array containing the image plus its label 
    new_array = (j, y_train[i])
    # Append the values to the array that will be shuffled
    X_train_shuffle.append(new_array)
    
# Have the new set of arrays
X_train_shuffle = np.array(X_train_shuffle)


In [ ]:
# Create the array for the test data set

X_test_shuffle = []

# It is necessary to create a for loop with enumeration as well
for i,j in enumerate(X_test):
    # The new array would be the array containing the image plus its label 
    new_array = (j, y_test[i])
    # Append the values to the array that will be shuffled
    X_test_shuffle.append(new_array)
    
# Have the new set of arrays  
X_test_shuffle = np.array(X_test_shuffle)

In [ ]:
# Apply the random shuffle to make the train and test with no specific order

np.random.shuffle(X_train_shuffle)

np.random.shuffle(X_test_shuffle)


In [ ]:
# Separate between the X_train and y_train to fit the model

X_train = []
y_train = []

# Start a for loop into the X_train_shuffle
for i in X_train_shuffle:
    # The array containing the picture would be the one that is in the index 0
    value = i[0]
    # The label would be the array that is on the index 1
    label = i[1]
    # Append the values and the labels to separate arrays
    X_train.append(value)
    y_train.append(label)

# Divide between X_train and y_train to run model
X_train = np.array(X_train)

y_train = np.array(y_train)

In [ ]:
# Same for the test data set

X_test = []
y_test = []

# Start a for loop into the X_test_shuffle
for i in X_test_shuffle:
    # The array containing the picture would be the one that is in the index 0
    value = i[0]
    # The label would be the array that is on the index 1
    label = i[1]
    # Append the values and the labels to separate arrays
    X_test.append(value)
    y_test.append(label)

X_test = np.array(X_test)

y_test = np.array(y_test)

In [ ]:
# Make sure labels are same than the first shapes

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# Define a function with the neural networks

def neural_network():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_initializer='he_uniform', 
                     padding = 'same', input_shape=(100, 70, 3)))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Fit model

model = neural_network()

# fit model
model.fit(X_train, y_train, epochs = 15)

In [ ]:
# Get the predictions

predictions = model.predict(X_test)

# Get them into 0 and 1 values

binary_values = []

# Start a for loop to iterate over the predictions array

for i in predictions:
    if i < 0.5:
        binary_values.append(0)
    if i >= 0.5:
        binary_values.append(1)
        

In [ ]:
# Display the confusion matrix to evaluate the model

cm = confusion_matrix(y_test, binary_values)
cmd = ConfusionMatrixDisplay(cm, display_labels=['not_flip','flip'])
cmd.plot()

plt.show()

In [ ]:
# Print the Classification report to get the precision, recall, f1-score

print(classification_report(y_test, binary_values))

### Conclusion

###### I was able to create a model with 0.99 accuray if a page needs to whether be flipped or not by using deep learning and doing the necessary data preprocessing such as making all the pages the same size, adjusting bright, adding nose, etc.

In [ ]:
# Save the model using pickle

model_classifier = model.save('flip_page_classifier')

# Video Creation

## Feature Engineering

In [ ]:
# Establish the training video by concatenating the flips with the not flips for train and test data sets.

#Establish the train video

video_1 = VideoFileClip('training_flip.avi')
video_2 = VideoFileClip('training_notflip.avi')

training_video = concatenate_videoclips([video_1, video_2])

training_video.write_videofile('training_video.avi', codec = 'rawvideo')
training_video.close()

In [ ]:
# Establish the test video

video_3 = VideoFileClip('test_flip.avi')
video_4 = VideoFileClip('test_notflip.avi')

test_video= concatenate_videoclips([video_3, video_4])

test_video.write_videofile('test_video.avi', codec = 'rawvideo')
test_video.close()

In [ ]:
# Separate the frames for the training video

count = 0

videoFile = 'training_video.avi'
# Capturing the video from the given path
cap = cv2.VideoCapture(videoFile)   
# Establish Frame rate
frameRate = cap.get(5) 

x=1

filenames_train = []

while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count;count+=1
        filenames_train.append(filename)
        cv2.imwrite(filename, frame)
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Separate the frames from the test video

count = 0

videoFile = 'test_video.avi'
# Capturing the video from the given path
cap = cv2.VideoCapture(videoFile)   
# Establish Frame rate
frameRate = cap.get(5) 

x=1

filenames_test = []

while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count;count+=1
        filenames_test.append(filename)
        cv2.imwrite(filename, frame)
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Append the y_train flip and not flip into the same array

y_train = np.append(np.array(y_train_flip), np.array(y_train_notflip))

# Append the y_test flip and not flip into the same array

y_test = np.append(np.array(y_test_flip), np.array(y_test_notflip))

# Create the data frame that will show the frameID and the class for the train data

data_train = pd.DataFrame({'frameID': filenames_train, 'flip': y_train})

# Create the data frame that will show the frameID and the class for the test data

data_test = pd.DataFrame({'frameID': filenames_test, 'flip': y_test})


In [ ]:
# Randomly change the data order and reset the index

data_train = shuffle(data_train).reset_index(drop = True)

data_test = shuffle(data_test).reset_index(drop = True)

In [ ]:
# Create an empty array

X_train = []   

# Loop through the frameID column and store every frame in X
for img_name in data_train.frameID:
    img = plt.imread('' + img_name)
    X_train.append(img)  
    
# Convert the list to an array
X_train = np.array(X_train)    

# Define the y_train

y_train = data_train['flip'].values

In [ ]:
# Create an empty array

X_test = []   

# Loop through the frameID column and store every frame in X
for img_name in data_test.frameID:
    img = plt.imread('' + img_name)
    X_test.append(img)  
    
# Convert the list to an array

X_test = np.array(X_test) 

# Define the y_test

y_test = data_test['flip'].values

In [ ]:
# Define a function to do the preprocessing for each frame of image of the video

def image_preprocessing_frame(data):
    # Create an empty list to store all the preprocessed images
    images = []
    # Start by creating a for loop through all the path and make the preprocessing to each image
    for i in data:
        # Adjust the size so all iamges will have the same size
        img = cv2.resize(i, dsize = (70,140), interpolation=cv2.INTER_CUBIC)
        # Crop to remove part of the images I don't need for the modeling part
        y,h,x,w = 0,100,0,70
        img = img[y:y+h, x:x+w]
        # Adjust brightness, contrast
        alpha=1.5
        beta=0.5
        img = cv2.addWeighted(img, alpha, np.zeros(img.shape, img.dtype), 0, beta)
        # Append the img to the list images
        images.append(img)
        # Create the video

    # Return the list with the preprocessed images
    return images

In [ ]:
# Use the defined function to preprocess the data

X_train = image_preprocessing_frame(data = X_train)

X_test = image_preprocessing_frame(data = X_test)

In [ ]:
# Include_top=False to remove the top layer and a base model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(100, 70, 3))    

In [ ]:
# Convert the list into arrays

X_train = np.array(X_train)

X_test = np.array(X_test)

In [ ]:
# We will make predictions using this model for X_train and X_valid, get the features, and then use those 
# features to retrain the model.

X_train = base_model.predict(X_train)

X_test = base_model.predict(X_test)

X_train.shape, X_test.shape

In [ ]:
# Centering the data

X_train = X_train/X_train.max()

X_test = X_test/X_test.max()

## Modeling

In [ ]:
# Define the neural network to use to predict if the frame of the video is flip or not flip

def model_neural():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_initializer='he_uniform', 
                     padding = 'same', input_shape=(3, 2, 512)))
    model.add(MaxPooling2D((2, 2)))
    
    model.add(Flatten())
    
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Use the function with the created model

model = model_neural()

model.fit(X_train, y_train, epochs = 100, validation_data = (X_test, y_test))

In [ ]:
# Get the predictions

predictions = model.predict(X_test)

# Get them into 0 and 1 values

binary_values = []

# Start a for loop to iterate over the predictions array

for i in predictions:
    if i < 0.5:
        binary_values.append(0)
    if i >= 0.5:
        binary_values.append(1)
        

## Evaluation of the Model

In [ ]:
# Display the confusion matrix to evaluate the model

cm = confusion_matrix(y_test, binary_values)
cmd = ConfusionMatrixDisplay(cm, display_labels=['not_flip','flip'])
cmd.plot()

plt.show()

In [ ]:
# Print the Classification report to get the precision, recall, f1-score

print(classification_report(y_test, binary_values))